In [40]:
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
%matplotlib inline
import seaborn as sn
import matplotlib.pyplot as plt
import sklearn.cluster
import distance

In [61]:
data=pd.read_excel('User_FInfo.xlsx',sheet_name='sheet_1')
data=data.fillna('')
data.sample(100)

,username,name,tweet,hashtags,Jobs,bio
1313,HaifaWehbe,Haifa Wehbe,['@paulina_kero @antoniosabd 4.8.2021 and this...,happyeaster,"singer , actress",
309,loyzagadiegs,Diego Loyzaga,['Do you like to play Play-To-Earn games? Get ...,"encountersagupaan,encountersabotahe,bekisonthe...","model , actor , video jockey",Actor. 🇵🇭🇪🇸🇦🇺 Only the strong survives IG: die...
950,Riteishd,Riteish Deshmukh,['Happy Birthday @nindusanj - have a great one...,"tokyoparalympics,thalaivi,sushilkumarshinde,si...","actor , television presenter , film producer","Proud Father, Devoted Husband, 14-Time World C..."
424,stephaniekelton,Stephanie Kelton,"['It took 48 hours, but we surpassed 500,000 v...","fridaynightzillow,thedeficitsthatmatter","economist , academic",@LFC | @JM7foundation | @NikeUK athlete | Inst...
690,alyssavaldez2,Alyssa Valdez,"['Good luck, Ej!! #ConfideceInTomorrow 💙', '@a...","confideceintomorrow,confidenceintomorrow,planf...",volleyball player,I LOVE therefore I AM ❤🙏🏻 https://t.co/GCqNBJ471L
...,...,...,...,...,...,...
1188,DjKingAssassin,Dj King Assassin,"[""We couldn't have the instruments no more in ...","theturntables,janetjackson,facts,djkingassassi...","disc jockey , producer , engineer","mompreneur, actress, writer, host, influencer|..."
211,layzhang,Lay Zhang,"[""Let's dance tonight in fashion^^ https://t....","lit_physical_album,producer,krump,peaceandlove...","rapper , singer , songwriter , record producer...",Music producer|Singer|Dancer|Actor| Email: inf...
380,pascrellfornj,"Bill Pascrell, Jr.",['What I remember most about that dark day 20 ...,"equalpay,laborday",politician,you found him. ✨
1281,KendraWilkinson,Kendra Wilkinson,"[""Sky's the limit. #KendraSellsHollywood http...","kendrasellshollywood,kendrasellshollywood,kend...","television personality , model",


In [47]:
titles =pd.Series(pd.Series(" ,".join(list(data['Jobs'].unique())).split(',')).unique())
titles=titles.str.strip().str.lower()
titles

0                       model
1      football wide receiver
2                      singer
3                     actress
4                       actor
                ...          
520                    singer
521                     judge
522          fashion designer
523     baritone opera singer
524            soprano singer
Length: 525, dtype: object

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
def plot_topn(sentences, ngram_range=(1,3), top=20,firstword=''):
    c=CountVectorizer(ngram_range=ngram_range)
    X=c.fit_transform(sentences)
    words=pd.DataFrame(X.sum(axis=0),columns=c.get_feature_names()).T.sort_values(0,ascending=False).reset_index()
    res=words[words['index'].apply(lambda x: firstword in x)].head(top)
    pl=px.bar(res, x='index',y=0)
    pl.update_layout(yaxis_title='count',xaxis_title='Phrases')
    pl.show()

In [73]:
import plotly.express as px
plot_topn(titles, ngram_range=(1,1))

In [50]:
data['tweet_bio'] = data['tweet'] + ' '+ data['bio']
data

,username,name,tweet,hashtags,Jobs,bio,tweet_bio
0,GiGiHadid,Gigi Hadid,"['few things 😋🙃😘', 'You were facing the right ...",giforce,model,IMG Models Worldwide. #giforce,"['few things 😋🙃😘', 'You were facing the right ..."
1,Cantguardmike,Michael William Thomas Jr.,"['Facts !', '@jhardee_19 called it like 2 mont...",gobucks,football wide receiver,GOD GOT US,"['Facts !', '@jhardee_19 called it like 2 mont..."
2,IAMJHUD,Jennifer Hudson,"['Team JHud, please help me wish my Virgo sist...","neverforget,baobabstudios,respectmovie,ebony,v...","singer , actress","Grammy winning Artist, Academy Award winning A...","['Team JHud, please help me wish my Virgo sist..."
3,lukajones,Luka Jones,"['@johnnypemberton Here we go 🏈', 'gracias for...","goclippers,rcjh,topcity,goblue,rcjh","actor , comedian",actor (Shrill & People of Earth both on @hulu)...,"['@johnnypemberton Here we go 🏈', 'gracias for..."
4,rcobb18,Randall Cobb (American football),['Man It Feels Good to Be a Packer https://t....,"mvp,ad,winitforted,jerseyswap,verzuz,ad,jersey...",football wide receiver,Faith in the Creator of Life. Husband. Father....,['Man It Feels Good to Be a Packer https://t....
...,...,...,...,...,...,...,...
1319,ColinHanks,Colin Hanks,['@PFTompkins The old man has a pretty sweet c...,"hankskerchiefs,hankskerchiefs,beready,beready,tbt","actor , producer , director",,['@PFTompkins The old man has a pretty sweet c...
1320,CraigyFerg,Craig Ferguson,['Tonight! @TheHustlerABC @ABCNetwork #thecrai...,"thecraigfergusonshow,thecraigfergusonshow,theh...","actor , author , comedian , television host",,['Tonight! @TheHustlerABC @ABCNetwork #thecrai...
1321,GeraldFinley,Gerald Finley,['@Twiz007 @DonaldMacleod01 @juliusdrake @edin...,"bluebeardscastle,otello,otello,dongiovanni,don...",baritone opera singer,,['@Twiz007 @DonaldMacleod01 @juliusdrake @edin...
1322,Emily_Estefan,Emily Estefan,['Just posted a photo https://t.co/ck0baQ3egI...,"damnitnova,davenavarro,imissmyband,purplemoney...",singer,,['Just posted a photo https://t.co/ck0baQ3egI...


In [41]:
embeddings_dict = {}
with open("glove.6B.100d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [43]:
def find_closest_embeddings(embedding, cutoff=25):
    return sorted(embeddings_dict.keys(), key=lambda token: spatial.distance.euclidean(embeddings_dict[token], embedding))

In [54]:
find_closest_embeddings(
    embeddings_dict["news"] + embeddings_dict["anchor"]
)[:5]

['news', 'anchor', 'cnn', 'television', 'abc']

In [55]:
find_closest_embeddings(
    embeddings_dict['worldwide']
)[:10]

['worldwide',
 'globally',
 'global',
 'europe',
 'world-wide',
 'nationwide',
 'addition',
 'increasing',
 'combined',
 'america']

In [51]:
lst=[]
for item in data['hashtags']:
    lst.extend(item.rsplit(','))
lst=list(set(lst))
lst.pop(0)

''

In [52]:
lst

['musicroyalties',
 'wentworth',
 'takemebackplease',
 'lit_physical_album',
 'childrensbooks',
 'vaxganda',
 'homedecor',
 'timeout2vote',
 'wonderfulindonesia',
 'culture3',
 'friday',
 'rsd2018',
 'warilapass',
 'harveymilk',
 'billionsclub',
 'superdancerchapter4',
 'communityshield',
 'b7',
 'education',
 'loveforsale',
 'smellreadyforanything',
 'besttelemamaondaplanet',
 'kisskissbangbang',
 'feedthenextbillion',
 'jimmorrison',
 'spacejam',
 'whereisgavin',
 'parisbercy',
 'remibader',
 'savortheater',
 'tcsummerjam',
 'fintechad',
 'mancrushmonday',
 'ryujin',
 'dontwaitup',
 'andy',
 'reno',
 'wcqualifiers',
 'facupfinal',
 'adizero',
 'goalietre',
 'currybrand',
 'paramveerchakara',
 'getsome',
 'themysteriousbenedictsociety',
 'totalbellas',
 'wearenxtuk',
 'respectthegame',
 'kinggayle',
 'rackcityday',
 'foodforthought',
 'wefoundshowtime',
 'ksfartistsofchoice',
 'zeecinemapremiere',
 'fearthedeer',
 'rallycircuit',
 'maddennfl22',
 'notourgame',
 'easter2020',
 'treatmy

In [62]:
data['embed']=data['Jobs'].apply(lambda x:x.rsplit(',')[:2]).apply(lambda x:[s.strip().lower().rsplit(' ') for s in x])
data['embed']

0                          [[model]]
1       [[football, wide, receiver]]
2              [[singer], [actress]]
3              [[actor], [comedian]]
4       [[football, wide, receiver]]
                    ...             
1319           [[actor], [producer]]
1320             [[actor], [author]]
1321     [[baritone, opera, singer]]
1322                      [[singer]]
1323             [[soprano, singer]]
Name: embed, Length: 1324, dtype: object

In [63]:
find_closest_embeddings(sum([embeddings_dict[item] for item in ['video', 'game', 'programmer']]))[0]

'game'

In [ ]:
sel=[0]*len(data)
for i, item in enumerate(data['embed']):
    try:
        sel[i]=[find_closest_embeddings(sum([embeddings_dict[item] for item in lst]))[0] for lst in item]
    except:
        sel[i]=[]
        print("Error arise from index {}".format(i))
    print(i)
            
data['embed']=pd.Series(sel)

Find and fix the null embeddings

In [ ]:
data['embed'][data['embed'].apply(lambda x:len(x))==0]

In [71]:
data.to_excel('User_embedding.xlsx', sheet_name='sheet1', index=False)

,username,name,tweet,hashtags,Jobs,bio,embed
0,GiGiHadid,Gigi Hadid,"['few things 😋🙃😘', 'You were facing the right ...",giforce,model,IMG Models Worldwide. #giforce,[[model]]
1,Cantguardmike,Michael William Thomas Jr.,"['Facts !', '@jhardee_19 called it like 2 mont...",gobucks,football wide receiver,GOD GOT US,"[[football, wide, receiver]]"
2,IAMJHUD,Jennifer Hudson,"['Team JHud, please help me wish my Virgo sist...","neverforget,baobabstudios,respectmovie,ebony,v...","singer , actress","Grammy winning Artist, Academy Award winning A...","[[singer], [actress]]"
3,lukajones,Luka Jones,"['@johnnypemberton Here we go 🏈', 'gracias for...","goclippers,rcjh,topcity,goblue,rcjh","actor , comedian",actor (Shrill & People of Earth both on @hulu)...,"[[actor], [comedian]]"
4,rcobb18,Randall Cobb (American football),['Man It Feels Good to Be a Packer https://t....,"mvp,ad,winitforted,jerseyswap,verzuz,ad,jersey...",football wide receiver,Faith in the Creator of Life. Husband. Father....,"[[football, wide, receiver]]"
...,...,...,...,...,...,...,...
1319,ColinHanks,Colin Hanks,['@PFTompkins The old man has a pretty sweet c...,"hankskerchiefs,hankskerchiefs,beready,beready,tbt","actor , producer , director",,"[[actor], [producer]]"
1320,CraigyFerg,Craig Ferguson,['Tonight! @TheHustlerABC @ABCNetwork #thecrai...,"thecraigfergusonshow,thecraigfergusonshow,theh...","actor , author , comedian , television host",,"[[actor], [author]]"
1321,GeraldFinley,Gerald Finley,['@Twiz007 @DonaldMacleod01 @juliusdrake @edin...,"bluebeardscastle,otello,otello,dongiovanni,don...",baritone opera singer,,"[[baritone, opera, singer]]"
1322,Emily_Estefan,Emily Estefan,['Just posted a photo https://t.co/ck0baQ3egI...,"damnitnova,davenavarro,imissmyband,purplemoney...",singer,,[[singer]]
